In [ ]:
#@markdown ⬅ 點擊執行按鈕開始分類
#@markdown ---
#@markdown <br><br>
#@markdown # 使用者設定
#@markdown ---
#@title 依擁有者分類資料夾下檔案 { display-mode: "form" }
欲分類資料夾 = "" #@param {type:"string", placeholder:"要分類的資料夾網址"}
#@markdown - 請輸入瀏覽器網址列顯示的網址。
#@markdown - 你必須是該資料夾的編輯者或擁有者。
#@markdown ---
儲存結果的父資料夾 ="" #@param {type:"string", placeholder:"儲存分類結果的父資料夾網址"}
#@markdown - 分類後的總資料夾將建立在此資料夾下。
#@markdown - 請輸入瀏覽器網址列顯示的網址。
#@markdown - 你必須是該資料夾的編輯者或擁有者。
#@markdown ---
總資料夾名稱 ="" #@param {type:"string", placeholder:"分類後建立的總資料夾名稱"}
#@markdown - 各帳號的資料夾將建立在此總資料夾下。
#@markdown ---
請求間隔 = 0.2 # @param {type:"slider", min:0.1, max:5, step:0.1}
#@markdown - API 請求之間的延遲（秒），以避免觸發速率限制。
#@markdown ---
# -*- coding: utf-8 -*-

# 變數名稱轉換
SOURCE_FOLDER_ID = 欲分類資料夾
DELAY_BETWEEN_REQUESTS = 請求間隔
DESTINATION_BASE_FOLDER_NAME = 總資料夾名稱
DESTINATION_ROOT_FOLDER_ID = 儲存結果的父資料夾

# --- 安裝與載入套件 ---
# !pip install --quiet --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib tqdm

from google.colab import auth, drive as colab_drive # 使用 google.colab.drive 掛載硬碟以便選擇資料夾 (可選)
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.auth import default
from tqdm.notebook import tqdm # 使用 notebook 版本的 tqdm
import time
import sys
from collections import deque # 用於遞迴處理

# --- 全域變數與統計 ---
stats = {
    'total_items_calculated': 0, # 預計處理的項目總數
    'total_files_processed': 0,  # 實際檢查的檔案數
    'total_files_moved': 0,      # 成功移動的檔案數
    'total_empty_folders_moved': 0, # 成功移動的空資料夾數
    'total_errors': 0,           # API 或其他錯誤數
    'skipped_already_in_target': 0 # 因已在目標位置而跳過的檔案/資料夾數
}
user_email = '' # 儲存執行腳本的使用者 Email
drive_service = None # Drive API 服務物件

# --- 輔助函數 ---

def authenticate_and_build_service():
    """處理 Colab 驗證並建立 Drive API 服務"""
    global drive_service, user_email
    try:
        auth.authenticate_user()
        creds, _ = default()
        drive_service = build('drive', 'v3', credentials=creds, cache_discovery=False) # 使用 v3 API
        # 取得使用者資訊以供後續擁有權檢查
        about = drive_service.about().get(fields="user").execute()
        user_email = about['user']['emailAddress']
        print(f"✅ 驗證成功，目前使用者: {user_email}")
        return True
    except Exception as e:
        print(f"❌ 驗證或建立服務失敗: {e}")
        return False

def get_or_create_folder_id(parent_folder_id, folder_name):
    """
    在指定的父資料夾中尋找或建立子資料夾，並返回其 ID。
    使用 Drive API v3。
    """
    global drive_service, DELAY_BETWEEN_REQUESTS
    # 檢查 folderName 是否有效
    if not folder_name or not isinstance(folder_name, str) or not folder_name.strip():
        print(f"  ❌ 無效的資料夾名稱: '{folder_name}'")
        raise ValueError(f"無效的資料夾名稱: '{folder_name}'")
    trimmed_folder_name = folder_name.strip()

    # 嘗試尋找現有的資料夾
    query = (f"'{parent_folder_id}' in parents and "
             f"name = '{trimmed_folder_name}' and "
             f"mimeType = 'application/vnd.google-apps.folder' and "
             f"trashed = false")
    try:
        time.sleep(DELAY_BETWEEN_REQUESTS)
        response = drive_service.files().list(q=query,
                                              spaces='drive',
                                              fields='files(id, name)',
                                              pageSize=1).execute()
        folders = response.get('files', [])
        if folders:
            # print(f"  ✅ 找到現有資料夾 '{folders[0]['name']}' (ID: {folders[0]['id']})")
            return folders[0]['id']
        else:
            # 資料夾不存在，建立新的
            print(f"  ➕ 在父資料夾 {parent_folder_id} 中建立資料夾 '{trimmed_folder_name}'...")
            folder_metadata = {
                'name': trimmed_folder_name,
                'mimeType': 'application/vnd.google-apps.folder',
                'parents': [parent_folder_id]
            }
            time.sleep(DELAY_BETWEEN_REQUESTS)
            new_folder = drive_service.files().create(body=folder_metadata,
                                                      fields='id, name').execute()
            print(f"  ✅ 成功建立資料夾 '{new_folder['name']}' (ID: {new_folder['id']})")
            return new_folder['id']
    except HttpError as error:
        print(f"  ❌ 尋找或建立資料夾 '{trimmed_folder_name}' 時發生 API 錯誤: {error}")
        raise error # 重新拋出錯誤，讓上層處理
    except Exception as e:
        print(f"  ❌ 處理資料夾 '{trimmed_folder_name}' 時發生非預期錯誤: {e}")
        raise e

def extract_folder_id_from_url(url_or_id):
    """嘗試從 Google Drive 資料夾 URL 中提取 ID。如果輸入看起來不像 URL，則直接返回。"""
    if not url_or_id:
        return None
    url_or_id = url_or_id.strip()
    if '/' in url_or_id and ('folders/' in url_or_id or '/drive/u/' in url_or_id):
        try:
            # 處理標準 folders/ URL
            if 'folders/' in url_or_id:
                folder_id = url_or_id.split('folders/')[1].split('?')[0].split('/')[0]
                return folder_id
            # 處理可能包含 /u/ 的 URL
            elif '/drive/' in url_or_id:
                 parts = url_or_id.split('/')
                 potential_id = parts[-1].split('?')[0]
                 if len(potential_id) > 20 and all(c.isalnum() or c in '-_' for c in potential_id):
                     return potential_id
            print(f"  ⚠️ 無法從提供的 URL 格式中提取 Folder ID: {url_or_id}")
            return None
        except IndexError:
            print(f"  ⚠️ 無法從提供的 URL 格式中提取 Folder ID: {url_or_id}")
            return None
    elif '/' not in url_or_id and ' ' not in url_or_id: # 簡單判斷是否可能是 ID
        return url_or_id
    else:
        print(f"  ⚠️ 輸入格式無法識別為有效的 URL 或 Folder ID: {url_or_id}")
        return None

def count_items_recursive(folder_id):
    """遞迴計算資料夾內的項目總數 (檔案 + 子資料夾)"""
    global drive_service, DELAY_BETWEEN_REQUESTS
    count = 0
    page_token = None
    while True:
        try:
            time.sleep(DELAY_BETWEEN_REQUESTS)
            param = {
                'q': f"'{folder_id}' in parents and trashed = false",
                'fields': "nextPageToken, files(id, mimeType)",
                'pageSize': 1000, # 盡量一次獲取多一點
                'pageToken': page_token,
                'supportsAllDrives': True, # 支持共用雲端硬碟
                'includeItemsFromAllDrives': True
            }
            response = drive_service.files().list(**param).execute()
            items = response.get('files', [])
            count += len(items) # 計算本層的檔案和資料夾

            # 對子資料夾進行遞迴計數
            for item in items:
                if item['mimeType'] == 'application/vnd.google-apps.folder':
                    count += count_items_recursive(item['id'])

            page_token = response.get('nextPageToken')
            if not page_token:
                break
        except HttpError as error:
            print(f"  ⚠️ 計算項目數量時讀取資料夾 {folder_id} 出錯: {error}")
            # 出錯時停止計數此分支，避免無限遞迴或錯誤擴大
            break
        except Exception as e:
            print(f"  ⚠️ 計算項目數量時發生非預期錯誤: {e}")
            break
    return count

# --- 核心處理函數 (遞迴版本) ---

def process_folder_recursively(folder_id, folder_name, base_dest_folder_id, pbar, is_source_folder=False):
    """
    遞迴處理資料夾內的檔案和子資料夾，並更新進度條 (DFS)。
    """
    global drive_service, stats, user_email, DELAY_BETWEEN_REQUESTS

    pbar.set_description(f"📁 {folder_name[:30]}...")

    # --- 1. 遞迴處理子資料夾 ---
    subfolders_to_process = []
    page_token_folders = None
    while True:
        try:
            time.sleep(DELAY_BETWEEN_REQUESTS)
            param_folders = {
                'q': f"'{folder_id}' in parents and mimeType = 'application/vnd.google-apps.folder' and trashed = false",
                'fields': "nextPageToken, files(id, name)",
                'pageSize': 100,
                'pageToken': page_token_folders,
                'supportsAllDrives': True,
                'includeItemsFromAllDrives': True
            }
            response_folders = drive_service.files().list(**param_folders).execute()
            subfolders = response_folders.get('files', [])
            subfolders_to_process.extend(subfolders) # 收集所有子資料夾

            page_token_folders = response_folders.get('nextPageToken')
            if not page_token_folders:
                break
        except HttpError as error:
            print(f"\n  ❌ 列出資料夾 '{folder_name}' 中的子資料夾時發生 API 錯誤: {error}")
            stats['total_errors'] += 1
            # 無法列出子資料夾，但仍嘗試處理本層檔案
            break
        except Exception as e:
            print(f"\n  ❌ 列出資料夾 '{folder_name}' 中的子資料夾時發生非預期錯誤: {e}")
            stats['total_errors'] += 1
            break

    # 對收集到的子資料夾進行遞迴呼叫
    for subfolder in subfolders_to_process:
        subfolder_id = subfolder['id']
        subfolder_name = subfolder.get('name', f'未命名資料夾 (ID: {subfolder_id})')
        process_folder_recursively(subfolder_id, subfolder_name, base_dest_folder_id, pbar)

    # --- 2. 處理目前資料夾中的檔案 (在子資料夾處理完畢後) ---
    page_token_files = None
    while True:
        try:
            time.sleep(DELAY_BETWEEN_REQUESTS)
            param_files = {
                'q': f"'{folder_id}' in parents and mimeType != 'application/vnd.google-apps.folder' and trashed = false",
                'fields': "nextPageToken, files(id, name, owners, parents)",
                'pageSize': 100,
                'pageToken': page_token_files,
                'supportsAllDrives': True,
                'includeItemsFromAllDrives': True
            }
            response_files = drive_service.files().list(**param_files).execute()
            files_in_folder = response_files.get('files', [])

            for file in files_in_folder:
                stats['total_files_processed'] += 1
                file_id = file['id']
                file_name = file.get('name', f'未命名檔案 (ID: {file_id})')
                pbar.set_postfix_str(f"📄 {file_name[:20]}", refresh=True)

                try:
                    owners = file.get('owners')
                    if not owners or not owners[0].get('emailAddress'):
                        print(f"\n    ⚠️ 跳過檔案 '{file_name}' (ID: {file_id})，無法取得擁有者資訊。")
                        stats['total_errors'] += 1
                        pbar.update(1) # 即使跳過也要更新進度
                        continue

                    owner_email = owners[0]['emailAddress']
                    owner_folder_name = owner_email
                    owner_dest_folder_id = get_or_create_folder_id(base_dest_folder_id, owner_folder_name)

                    current_parents = file.get('parents', [])
                    if owner_dest_folder_id in current_parents:
                        stats['skipped_already_in_target'] += 1
                        pbar.update(1)
                        continue
                    elif folder_id not in current_parents:
                        # 檔案可能已被移動或權限變更，記錄但不視為嚴重錯誤
                        print(f"\n    ❓ 檔案 '{file_name}' (ID: {file_id}) 不在預期來源 '{folder_name}' 中，跳過。")
                        pbar.update(1)
                        continue
                    else:
                        time.sleep(DELAY_BETWEEN_REQUESTS)
                        drive_service.files().update(
                            fileId=file_id,
                            addParents=owner_dest_folder_id,
                            removeParents=folder_id,
                            fields='id', # 只需要知道成功即可
                            supportsAllDrives=True
                        ).execute()
                        stats['total_files_moved'] += 1
                        pbar.update(1) # 移動成功後更新進度

                except HttpError as error:
                    stats['total_errors'] += 1
                    print(f"\n    ❌ 處理檔案 '{file_name}' (ID: {file_id}) 時 API 錯誤: {error}")
                    pbar.update(1) # 出錯也要更新進度
                except ValueError as e: # 捕捉 get_or_create_folder_id 的錯誤
                     stats['total_errors'] += 1
                     print(f"\n    ❌ 處理檔案 '{file_name}' 時目標資料夾名稱錯誤: {e}")
                     pbar.update(1)
                except Exception as e:
                    stats['total_errors'] += 1
                    print(f"\n    ❌ 處理檔案 '{file_name}' (ID: {file_id}) 時非預期錯誤: {e}")
                    pbar.update(1) # 出錯也要更新進度

            page_token_files = response_files.get('nextPageToken')
            if not page_token_files:
                break
        except HttpError as error:
            print(f"\n  ❌ 列出資料夾 '{folder_name}' 中的檔案時發生 API 錯誤: {error}")
            stats['total_errors'] += 1
            # 無法列出檔案，但還是嘗試處理資料夾本身
            break
        except Exception as e:
            print(f"\n  ❌ 列出資料夾 '{folder_name}' 中的檔案時發生非預期錯誤: {e}")
            stats['total_errors'] += 1
            break

    # --- 3. 檢查目前資料夾是否為空並嘗試移動 (在子資料夾和檔案都處理完畢後) ---
    folder_processed_or_skipped = False # 標記此資料夾是否已被處理 (移動/跳過/錯誤)
    if not is_source_folder: # 不處理根來源資料夾
        try:
            # 重新檢查資料夾內容是否為空
            time.sleep(DELAY_BETWEEN_REQUESTS)
            param_check_empty = {
                'q': f"'{folder_id}' in parents and trashed = false",
                'fields': "files(id)", 'pageSize': 1,
                'supportsAllDrives': True, 'includeItemsFromAllDrives': True
            }
            response_check = drive_service.files().list(**param_check_empty).execute()
            items_inside = response_check.get('files', [])

            if not items_inside:
                # 資料夾是空的，嘗試移動
                time.sleep(DELAY_BETWEEN_REQUESTS)
                folder_meta = drive_service.files().get(fileId=folder_id, fields='owners, trashed, parents', supportsAllDrives=True).execute()
                folder_owners = folder_meta.get('owners')
                is_trashed = folder_meta.get('trashed', False)
                current_parents = folder_meta.get('parents')

                if is_trashed:
                    folder_processed_or_skipped = True # 已在垃圾桶，視為已處理
                elif not folder_owners or not folder_owners[0].get('emailAddress'):
                    print(f"\n  ⚠️ 跳過移動空資料夾 '{folder_name}'，無法取得擁有者。")
                    stats['total_errors'] += 1
                    folder_processed_or_skipped = True # 標記為已處理 (因錯誤跳過)
                elif not current_parents:
                     print(f"\n  ⚠️ 跳過移動空資料夾 '{folder_name}'，無法取得父資料夾。")
                     stats['total_errors'] += 1
                     folder_processed_or_skipped = True # 標記為已處理 (因錯誤跳過)
                else:
                    owner_email = folder_owners[0]['emailAddress']
                    owner_folder_name = owner_email
                    try:
                        owner_dest_folder_id = get_or_create_folder_id(base_dest_folder_id, owner_folder_name)
                        if owner_dest_folder_id not in current_parents:
                            parent_to_remove = current_parents[0] # 假設第一個是來源樹的父節點
                            time.sleep(DELAY_BETWEEN_REQUESTS)
                            drive_service.files().update(
                                fileId=folder_id,
                                addParents=owner_dest_folder_id,
                                removeParents=parent_to_remove,
                                fields='id',
                                supportsAllDrives=True
                            ).execute()
                            stats['total_empty_folders_moved'] += 1
                            folder_processed_or_skipped = True # 標記已移動
                        else:
                            stats['skipped_already_in_target'] += 1 # 已在目標位置
                            folder_processed_or_skipped = True # 視為已處理

                    except HttpError as error_move:
                        stats['total_errors'] += 1
                        print(f"\n    ❌ 移動空資料夾 '{folder_name}' 時 API 錯誤: {error_move}")
                        folder_processed_or_skipped = True # 標記為已處理 (因錯誤跳過)
                    except ValueError as e_move: # 捕捉 get_or_create_folder_id 的錯誤
                        stats['total_errors'] += 1
                        print(f"\n    ❌ 移動空資料夾 '{folder_name}' 時目標資料夾名稱錯誤: {e_move}")
                        folder_processed_or_skipped = True # 標記為已處理 (因錯誤跳過)
                    except Exception as e_move:
                        stats['total_errors'] += 1
                        print(f"\n    ❌ 移動空資料夾 '{folder_name}' 時非預期錯誤: {e_move}")
                        folder_processed_or_skipped = True # 標記為已處理 (因錯誤跳過)

        except HttpError as error:
            stats['total_errors'] += 1
            print(f"\n  ❌ 檢查或移動空資料夾 '{folder_name}' 時 API 錯誤: {error}")
            folder_processed_or_skipped = True # 標記為已處理 (因錯誤跳過)
        except Exception as e:
            stats['total_errors'] += 1
            print(f"\n  ❌ 檢查或移動空資料夾 '{folder_name}' 時非預期錯誤: {e}")
            folder_processed_or_skipped = True # 標記為已處理 (因錯誤跳過)

    # --- 4. 更新資料夾本身的進度 ---
    # 如果資料夾被成功移動、跳過或因錯誤無法移動，則更新進度
    # 如果資料夾未被處理（例如非空、是根目錄），也更新進度
    pbar.update(1)
    pbar.set_postfix_str("") # 清除檔案名

# --- 主執行流程 ---

if __name__ == "__main__":
    print("🚀 開始執行檔案擁有者分類腳本 🚀")
    start_time_total = time.time() # 記錄總開始時間

    if not authenticate_and_build_service():
        sys.exit("無法繼續執行，請檢查驗證問題。")

    # --- 確定來源資料夾 ID ---
    source_input = SOURCE_FOLDER_ID
    source_folder_id = extract_folder_id_from_url(source_input)
    manual_input = None # 初始化 manual_input
    if not source_folder_id:
        print("\n⚠️ 未在設定中指定有效的來源資料夾 URL 或 ID (SOURCE_FOLDER_ID)。")
        # 互動式輸入在 Colab 中可能不穩定，優先使用參數
        # manual_input = input("👉 請手動輸入您想處理的來源資料夾 URL 或 ID：").strip()
        # source_folder_id = extract_folder_id_from_url(manual_input)
        if not source_folder_id: # 如果參數和手動輸入都無效
            sys.exit("❌ 未提供有效的來源資料夾 URL 或 ID，無法繼續執行。")

    # 驗證來源資料夾
    try:
        print(f"\n🔍 正在驗證來源資料夾 (來自輸入: '{source_input or manual_input}')...")
        time.sleep(DELAY_BETWEEN_REQUESTS)
        source_folder_info = drive_service.files().get(fileId=source_folder_id, fields="id, name, mimeType", supportsAllDrives=True).execute()
        if source_folder_info.get('mimeType') != 'application/vnd.google-apps.folder':
             sys.exit(f"❌ 提供的 ID '{source_folder_id}' 不是一個資料夾。")
        source_folder_name = source_folder_info.get('name', '未命名來源資料夾')
        print(f"✅ 來源資料夾確認: '{source_folder_name}' (ID: {source_folder_id})")
    except HttpError as error:
        sys.exit(f"❌ 無法存取來源資料夾 ID '{source_folder_id}'，請檢查 URL/ID 及權限: {error}")
    except Exception as e:
        sys.exit(f"❌ 驗證來源資料夾時發生錯誤: {e}")

    # --- 確定目標資料夾結構 ---
    dest_root_input = DESTINATION_ROOT_FOLDER_ID
    dest_root_folder_id = extract_folder_id_from_url(dest_root_input)
    if not dest_root_folder_id:
         sys.exit(f"❌ 設定的目標父資料夾 URL 或 ID '{dest_root_input}' 無效或無法提取 ID。")

    try:
        print(f"\n🎯 正在設定目標資料夾結構...")
        # 驗證目標父資料夾
        print(f"  🔍 驗證目標父資料夾 (來自輸入: '{dest_root_input}')...")
        time.sleep(DELAY_BETWEEN_REQUESTS)
        dest_root_info = drive_service.files().get(fileId=dest_root_folder_id, fields="id, name, mimeType, capabilities", supportsAllDrives=True).execute()
        if dest_root_info.get('mimeType') != 'application/vnd.google-apps.folder':
             sys.exit(f"❌ 設定的目標父資料夾 ID '{dest_root_folder_id}' 不是一個資料夾。")
        if not dest_root_info.get('capabilities', {}).get('canAddChildren'):
             sys.exit(f"❌ 您沒有在目標父資料夾 '{dest_root_info.get('name')}' 中建立資料夾的權限。")
        dest_root_name = dest_root_info.get('name', '未命名目標父資料夾')
        print(f"  ✅ 目標父資料夾確認: '{dest_root_name}' (ID: {dest_root_folder_id})")

        # 取得或建立總資料夾
        base_dest_folder_id = get_or_create_folder_id(dest_root_folder_id, DESTINATION_BASE_FOLDER_NAME)
        print(f"✅ 使用總資料夾 '{DESTINATION_BASE_FOLDER_NAME}' (ID: {base_dest_folder_id})")

    except HttpError as error:
        sys.exit(f"❌ 無法存取或建立目標資料夾結構，請檢查目標根 URL/ID '{dest_root_folder_id}' 和權限: {error}")
    except ValueError as e: # 捕捉 get_or_create_folder_id 拋出的無效名稱錯誤
         sys.exit(f"❌ 設定目標資料夾時發生錯誤: {e}")
    except Exception as e:
        sys.exit(f"❌ 設定目標資料夾時發生錯誤: {e}")

    # --- 計算總項目數 ---
    print(f"\n⏳ 正在計算來源資料夾 '{source_folder_name}' 中的項目總數 (這可能需要一些時間)...")
    start_count_time = time.time()
    total_items_to_process = 0
    try:
        # 計算來源資料夾 *內部* 的所有項目
        count_inside = count_items_recursive(source_folder_id)
        # 總數 = 內部項目數 + 頂層資料夾本身 (1)
        total_items_to_process = count_inside + 1
        stats['total_items_calculated'] = total_items_to_process # 記錄預計總數
        count_duration = time.time() - start_count_time
        print(f"📊 計算完成，共需處理約 {total_items_to_process} 個項目 (包含檔案和資料夾)。(耗時 {count_duration:.2f} 秒)")
        if total_items_to_process == 1 and count_inside == 0:
            print(f"  ℹ️ 來源資料夾 '{source_folder_name}' 為空。")
    except Exception as e:
        print(f"\n❌ 計算項目總數時發生嚴重錯誤: {e}")
        print("   無法繼續執行。")
        sys.exit()

    # --- 開始遞迴處理 ---
    if total_items_to_process > 0: # 只有在有項目時才處理
        print("\n⏳ 開始處理項目...")
        start_process_time = time.time()

        # 使用 tqdm 創建進度條
        bar_format = '{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_fmt}{postfix}]'
        with tqdm(total=total_items_to_process, desc="準備中...", unit="項", bar_format=bar_format) as pbar:
            try:
                # 呼叫新的遞迴函數，並標記這是來源資料夾
                process_folder_recursively(source_folder_id, source_folder_name, base_dest_folder_id, pbar, is_source_folder=True)
            except Exception as e:
                print(f"\n🚨 處理過程中發生嚴重錯誤: {e}")
                stats['total_errors'] += 1 # 將此類錯誤計入總數
                # 即使出錯，也嘗試關閉進度條
                pbar.close() # 確保進度條關閉
            finally:
                 # 確保進度條在正常結束或異常結束時都能填滿 (如果尚未完成)
                 # 但要注意，如果中途嚴重錯誤，計數可能不准
                 if pbar.n < pbar.total:
                     pbar.n = pbar.total # 手動設置為完成狀態
                     pbar.refresh() # 刷新顯示
                 pbar.set_description("✅ 處理完成")
                 pbar.set_postfix_str("")


        process_duration = time.time() - start_process_time
    else:
        print("\n🤷 來源資料夾為空或無法計算項目，無需處理。")
        process_duration = 0

    total_duration = time.time() - start_time_total

    # --- 顯示最終統計結果 ---
    print("\n🏁 ========== 處理完成 ========== 🏁")
    print(f"⏱️ 處理耗時: {process_duration:.2f} 秒 (總耗時: {total_duration:.2f} 秒)")
    print(f"🌳 來源資料夾: '{source_folder_name}' (ID: {source_folder_id})")
    print(f"🎯 總資料夾: '{DESTINATION_BASE_FOLDER_NAME}' (位於 '{dest_root_name}' 下)")
    print("-" * 30)
    print(f"📊 統計結果:")
    print(f"  - 預計處理項目數: {stats['total_items_calculated']}")
    print(f"  - 實際檢查的檔案數: {stats['total_files_processed']}")
    print(f"  - 成功移動的檔案數: {stats['total_files_moved']}")
    print(f"  - 成功移動的空資料夾數: {stats['total_empty_folders_moved']}")
    print(f"  - 因已在目標位置而跳過數: {stats['skipped_already_in_target']}")
    print(f"  - 發生錯誤/其他跳過數: {stats['total_errors']}")
    processed_count_approx = stats['total_files_moved'] + stats['total_empty_folders_moved'] + stats['skipped_already_in_target'] + stats['total_errors']
    print(f"  - (約略已處理/跳過/錯誤總計: {processed_count_approx})")
    print("===================================")

# 腳本說明：依擁有者分類資料夾下檔案

## 用途

*   此腳本會自動整理 Google Drive 中指定「欲分類資料夾」內的檔案與子資料夾。
*   它會檢查每個檔案的**擁有者 (Owner)**。
*   根據檔案擁有者的 Email，將檔案移動到指定的「儲存結果的父資料夾」下的一個新建立的「總資料夾名稱」（例如 "待刪除檔案"）中，並以擁有者的 Email 命名子資料夾（例如 `待刪除檔案/user1@example.com/`）。
*   處理過程中，如果原始的子資料夾因為內部的檔案都被移走而變空，腳本會嘗試將這個空的子資料夾也移動到其擁有者對應的目標 Email 資料夾下。

## 適用情境

*   當一個共享資料夾內混合了多位使用者擁有的檔案，需要將檔案歸類整理給各自的擁有者時。
*   作為清理共享空間的前置步驟，讓每個使用者可以方便地找到並處理自己擁有的檔案。
*   自動化整理大量檔案，避免手動逐一檢查和移動的繁瑣工作。

## 權限要求

*   **執行者**：執行此腳本的 Google 帳號需要：
    *   對「欲分類資料夾」擁有至少**編輯者 (Editor)** 權限，才能讀取內容並移動其中的項目。
    *   對「儲存結果的父資料夾」擁有至少**編輯者 (Editor)** 權限，才能在其中建立「總資料夾名稱」以及後續的 Email 子資料夾。
*   **檔案/資料夾**：
    *   腳本需要能夠讀取每個檔案和子資料夾的擁有者資訊。
    *   對於要移動的檔案或空的子資料夾，執行者需要有權限修改其父資料夾（即將其從來源移出，並加入到目標）。

## 警告說明

*   **移動操作**：此腳本會**實際移動**檔案和空的子資料夾，改變它們在 Google Drive 中的位置。請確保您了解此操作的後果。
*   **目標資料夾結構**：腳本會在指定的「儲存結果的父資料夾」下建立新的資料夾結構。請確認目標位置正確。
*   **權限問題**：如果執行者對某些檔案或子資料夾缺乏足夠的移動權限，或者無法讀取擁有者資訊，這些項目將被跳過並記錄錯誤。
*   **API 限制**：處理大量檔案時，可能會遇到 Google Drive API 的速率限制或超時。腳本內建有延遲機制，但錯誤仍可能發生。建議分批處理超大資料夾。
*   **空資料夾處理**：只有當子資料夾內所有檔案都被成功移走後，腳本才會嘗試移動這個空的子資料夾。如果移動檔案過程中出錯，或資料夾原本就包含無法處理的項目，則該子資料夾可能不會被移動。
*   **共用雲端硬碟 (Shared Drive)**：腳本嘗試支援共用雲端硬碟 (`supportsAllDrives=True`)，但共用雲端硬碟的權限模型與「我的雲端硬碟」不同，移動操作的行為和權限要求可能會有差異。
*   **執行時間**：處理大量檔案和深層資料夾結構可能需要很長時間。腳本會顯示進度條，但請耐心等待。